## FINITE DIFFERENCING

Finite differencing of the potential is a method of approximating the derivative of the potential (the force) at a certain point. The gradient of the potential can be found via a simple $dy/dx$ where $dy$ is the change in potential and $dx$ is a small change in position. This method should reproduce the force exactly as $dx \rightarrow 0$.

First, a system should displaced away from equilibrium, such that there is potential energy and hence a force that will act upon the beads in the model. This position should then be perturbed in each direction and the change in potential calculated to approximate the force at this displacement.

These values of force can then be compared against those obtained via analytical differentiation of the potential energy. This is a method of validating the differentiation of the potential and ensuring the forces will always be correctly calculated. 

First, it will be constructive to carry this procedure out for the simple case of stretching forces. 

In [40]:
import numpy as np

In [41]:
r0 = 0.5 #distance between beads (rest length of spring)

In [42]:
def spring(A,B):
    """
    This function returns the stretching force on bead A as a result of the 
    position of bead B. The returned forces have units of k, the stretching
    modulus.
    """
    AB = np.subtract(B,A)
    r = np.linalg.norm(AB)
    prefactor = (r - r0) / r
    
    Fx = prefactor * AB[0]
    Fy = prefactor * AB[1]
    Fz = prefactor * AB[2]
    
    return np.array([Fx,Fy,Fz])

The above function calculates the stretching forces that act upon a bead A due to the position of bead B having stretched the spring. 

In [43]:
X = np.array([0,0,0])
Y = np.array([0.5,0,0])
no_extension = spring(X,Y)
print("At no extension, the force on bead A is: {}".format(no_extension))

Y = np.array([0.5,0,0.02])
perturbed = spring(X,Y)
print("Once the spring has been extended, there is a force on bead A of: {}".format(perturbed))


At no extension, the force on bead A is: [0. 0. 0.]
Once the spring has been extended, there is a force on bead A of: [3.99520639e-04 0.00000000e+00 1.59808256e-05]


In [44]:
def finite_differencing_spring(displacement):
    """
    This function should perform finite differencing of the potential to 
    confirm the results for force on bead A returned by the spring() function.
    The displacement inputted should be the purterbation to bead B's position.
    """
    
    # Initial positions
    A = np.array([0, 0, 0])
    B = np.array([r0, 0, 0])
    
    # Apply displacement to B
    for i in range(3):
        B[i] = B[i] + displacement[i]
        
    AB = np.subtract(B,A)
    AB_mag = np.linalg.norm(AB)
    
    # Potential after displacement
    V = 0.5 * (AB_mag - r0)**2

    # Compute forces after small perturbation (gradient at that point)
    delta_x = 1e-6
    V_perturbed = 0.5 * ((np.linalg.norm(B + np.array([delta_x, 0, 0]) - A) - r0)**2)
    F_x = (V_perturbed - V) / delta_x
    
    delta_y = 1e-6
    V_perturbed = 0.5 * ((np.linalg.norm(B + np.array([0, delta_y, 0]) - A) - r0)**2)
    F_y = (V_perturbed - V) / delta_y
    
    delta_z = 1e-6
    V_perturbed = 0.5 * ((np.linalg.norm(B + np.array([0, 0, delta_z]) - A) - r0)**2)
    F_z = (V_perturbed - V) / delta_z
    
    return np.array([F_x, F_y, F_z])

In [45]:
displacement = np.array([0,0,0.02])

force = finite_differencing_spring(displacement)
print("Via analytical differentiation:")
print(perturbed)
print("\n")

print("Via finite differencing: ")
print(force)


Via analytical differentiation:
[3.99520639e-04 0.00000000e+00 1.59808256e-05]


Via finite differencing: 
[4.00019841e-04 3.99476157e-10 1.59820232e-05]


As can be seen above, finite differencing has well approximated the force at this displacement. The perturbation used was 1e-8. Now to perform the same procudure for the bending forces. Firstly, we can write the alaytical derivative of the bending potential in terms of the coordinates of the beads:

In [46]:
def unit_cell_bending_force(I,J,K,L):
    """
    From the positions of four beads, this function calculates the bending
    force that each bead experiences as a result of the presence of the other
    beads in the unit cell IJKL, where I and K are the central beads in the 
    cell (they have three connections, whereas J and L have only two).
    
    The returned forces have units of κ.
    """
    
    Ix = I[0]
    Iy = I[1]
    Iz = I[2]
    
    Jx = J[0]
    Jy = J[1]
    Jz = J[2]

    Kx = K[0]
    Ky = K[1]
    Kz = K[2]
    
    Lx = L[0]
    Ly = L[1]
    Lz = L[2]
    
    ddIx = ( -1/2 * ( ( ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 \
    ) + ( ( ( -1 * Iz + Jz ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( \
    ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( ( ( -1 * \
    Iz + Jz ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( -2 * ( ( -1 * Ix \
    + Jx ) * ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( Jy + -1 * Ky ) + ( ( \
    -1 * Ix + Jx ) * ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( Jz + -1 * Kz \
    ) + ( -1 * Iy + Jy ) * ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( Jz + \
    -1 * Kz ) ) ) + ( ( ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jz + -1 * Kz ) \
    )**( 2 ) + ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) \
    ) ) ) ) ) ) )**( -1/2 ) * ( ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( \
    -1 * Kx + Lx ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Ix + Jx ) * ( Jy + \
    -1 * Ky ) * ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Iy + \
    Jy ) * ( Jx + -1 * Kx ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly ) + ( ( \
    -1 * Ix + Jx ) * ( Jy + -1 * Ky ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly \
    ) + ( ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( -1 * Kx + Lx ) * ( -1 * \
    Iz + Lz ) + ( -1 * ( -1 * Ix + Jx ) * ( Jz + -1 * Kz ) * ( -1 * Kx + \
    Lx ) * ( -1 * Iz + Lz ) + ( ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( \
    -1 * Ky + Ly ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iy + Jy ) * ( Jz + \
    -1 * Kz ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iz + \
    Jz ) * ( Jx + -1 * Kx ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz ) + ( ( \
    -1 * Ix + Jx ) * ( Jz + -1 * Kz ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz \
    ) + ( -1 * ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( -1 * Iy + Ly ) * ( \
    -1 * Kz + Lz ) + ( -1 * Iy + Jy ) * ( Jz + -1 * Kz ) * ( -1 * Iy + Ly \
    ) * ( -1 * Kz + Lz ) ) ) ) ) ) ) ) ) ) ) ) * ( -2 * ( -1 * Ix + Lx ) \
    * ( ( -1 * Ky + Ly ) )**( 2 ) + ( -2 * ( -1 * Ix + Lx ) * ( ( -1 * Kz \
    + Lz ) )**( 2 ) + -2 * ( -1 * ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) * ( \
    -1 * Ky + Ly ) + -1 * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) * ( -1 * Kz \
    + Lz ) ) ) ) * ( ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iy + Ly ) \
    )**( 2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * Ky + Ly ) )**( 2 \
    ) + ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( ( \
    ( -1 * Ky + Ly ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( ( ( -1 * \
    Ix + Lx ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + ( ( ( -1 * Iy + Ly \
    ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + -2 * ( ( -1 * Ix + Lx ) * \
    ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) + ( ( -1 * Ix \
    + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) * ( -1 * Kz + Lz ) + ( \
    -1 * Iy + Ly ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) * ( -1 * Kz + Lz \
    ) ) ) ) ) ) ) ) ) )**( -3/2 ) + ( ( ( ( ( -1 * Iy + Jy ) )**( 2 ) * ( \
    ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( -1 * Iz + Jz ) )**( 2 ) * ( ( Jx + \
    -1 * Kx ) )**( 2 ) + ( ( ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jy + -1 * Ky \
    ) )**( 2 ) + ( ( ( -1 * Iz + Jz ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( \
    2 ) + ( -2 * ( ( -1 * Ix + Jx ) * ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) \
    * ( Jy + -1 * Ky ) + ( ( -1 * Ix + Jx ) * ( -1 * Iz + Jz ) * ( Jx + \
    -1 * Kx ) * ( Jz + -1 * Kz ) + ( -1 * Iy + Jy ) * ( -1 * Iz + Jz ) * \
    ( Jy + -1 * Ky ) * ( Jz + -1 * Kz ) ) ) + ( ( ( -1 * Ix + Jx ) )**( 2 \
    ) * ( ( Jz + -1 * Kz ) )**( 2 ) + ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( \
    Jz + -1 * Kz ) )**( 2 ) ) ) ) ) ) ) )**( -1/2 ) * ( ( Jy + -1 * Ky ) \
    * ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) + ( ( -1 * Iy + Jy ) * ( Jx + \
    -1 * Kx ) * ( -1 * Ky + Ly ) + ( -1 * ( -1 * Ix + Jx ) * ( Jy + -1 * \
    Ky ) * ( -1 * Ky + Ly ) + ( -1 * ( Jy + -1 * Ky ) * ( -1 * Ix + Lx ) \
    * ( -1 * Ky + Ly ) + ( ( Jz + -1 * Kz ) * ( -1 * Kx + Lx ) * ( -1 * \
    Iz + Lz ) + ( ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( -1 * Kz + Lz ) \
    + ( -1 * ( -1 * Ix + Jx ) * ( Jz + -1 * Kz ) * ( -1 * Kz + Lz ) + -1 \
    * ( Jz + -1 * Kz ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz ) ) ) ) ) ) ) \
    ) * ( ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iy + Ly ) )**( 2 ) + ( \
    ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * Ky + Ly ) )**( 2 ) + ( ( ( -1 \
    * Kx + Lx ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( ( ( -1 * Ky + \
    Ly ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( ( ( -1 * Ix + Lx ) \
    )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + ( ( ( -1 * Iy + Ly ) )**( 2 \
    ) * ( ( -1 * Kz + Lz ) )**( 2 ) + -2 * ( ( -1 * Ix + Lx ) * ( -1 * Kx \
    + Lx ) * ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) + ( ( -1 * Ix + Lx ) * ( \
    -1 * Kx + Lx ) * ( -1 * Iz + Lz ) * ( -1 * Kz + Lz ) + ( -1 * Iy + Ly \
    ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) * ( -1 * Kz + Lz ) ) ) ) ) ) \
    ) ) ) )**( -1/2 ) + -1/2 * ( -2 * ( -1 * Ix + Jx ) * ( ( Jy + -1 * Ky \
    ) )**( 2 ) + ( -2 * ( -1 * ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( Jy \
    + -1 * Ky ) + -1 * ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( Jz + -1 * \
    Kz ) ) + -2 * ( -1 * Ix + Jx ) * ( ( Jz + -1 * Kz ) )**( 2 ) ) ) * ( \
    ( ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( \
    -1 * Iz + Jz ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( -1 * Ix \
    + Jx ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( ( ( -1 * Iz + Jz ) \
    )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( -2 * ( ( -1 * Ix + Jx ) * \
    ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( Jy + -1 * Ky ) + ( ( -1 * Ix \
    + Jx ) * ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( Jz + -1 * Kz ) + ( \
    -1 * Iy + Jy ) * ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( Jz + -1 * Kz \
    ) ) ) + ( ( ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) + \
    ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) ) ) ) ) ) ) \
    )**( -3/2 ) * ( ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( -1 * Kx + Lx \
    ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Ix + Jx ) * ( Jy + -1 * Ky ) * ( \
    -1 * Kx + Lx ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Iy + Jy ) * ( Jx + \
    -1 * Kx ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly ) + ( ( -1 * Ix + Jx ) \
    * ( Jy + -1 * Ky ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly ) + ( ( -1 * \
    Iz + Jz ) * ( Jx + -1 * Kx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) + \
    ( -1 * ( -1 * Ix + Jx ) * ( Jz + -1 * Kz ) * ( -1 * Kx + Lx ) * ( -1 \
    * Iz + Lz ) + ( ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( -1 * Ky + Ly \
    ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iy + Jy ) * ( Jz + -1 * Kz ) * ( \
    -1 * Ky + Ly ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iz + Jz ) * ( Jx + \
    -1 * Kx ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz ) + ( ( -1 * Ix + Jx ) \
    * ( Jz + -1 * Kz ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz ) + ( -1 * ( \
    -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( -1 * Iy + Ly ) * ( -1 * Kz + Lz \
    ) + ( -1 * Iy + Jy ) * ( Jz + -1 * Kz ) * ( -1 * Iy + Ly ) * ( -1 * \
    Kz + Lz ) ) ) ) ) ) ) ) ) ) ) ) * ( ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( \
    ( -1 * Iy + Ly ) )**( 2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * \
    Ky + Ly ) )**( 2 ) + ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iz + Lz \
    ) )**( 2 ) + ( ( ( -1 * Ky + Ly ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( \
    2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + ( \
    ( ( -1 * Iy + Ly ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + -2 * ( ( \
    -1 * Ix + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) * ( -1 * Ky + Ly \
    ) + ( ( -1 * Ix + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) * ( -1 * \
    Kz + Lz ) + ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) * \
    ( -1 * Kz + Lz ) ) ) ) ) ) ) ) ) )**( -1/2 ) ) )
    
    ddIy = ( -1/2 * ( ( ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 \
    ) + ( ( ( -1 * Iz + Jz ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( \
    ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( ( ( -1 * \
    Iz + Jz ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( -2 * ( ( -1 * Ix \
    + Jx ) * ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( Jy + -1 * Ky ) + ( ( \
    -1 * Ix + Jx ) * ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( Jz + -1 * Kz \
    ) + ( -1 * Iy + Jy ) * ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( Jz + \
    -1 * Kz ) ) ) + ( ( ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jz + -1 * Kz ) \
    )**( 2 ) + ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) \
    ) ) ) ) ) ) )**( -1/2 ) * ( ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( \
    -1 * Kx + Lx ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Ix + Jx ) * ( Jy + \
    -1 * Ky ) * ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Iy + \
    Jy ) * ( Jx + -1 * Kx ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly ) + ( ( \
    -1 * Ix + Jx ) * ( Jy + -1 * Ky ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly \
    ) + ( ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( -1 * Kx + Lx ) * ( -1 * \
    Iz + Lz ) + ( -1 * ( -1 * Ix + Jx ) * ( Jz + -1 * Kz ) * ( -1 * Kx + \
    Lx ) * ( -1 * Iz + Lz ) + ( ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( \
    -1 * Ky + Ly ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iy + Jy ) * ( Jz + \
    -1 * Kz ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iz + \
    Jz ) * ( Jx + -1 * Kx ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz ) + ( ( \
    -1 * Ix + Jx ) * ( Jz + -1 * Kz ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz \
    ) + ( -1 * ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( -1 * Iy + Ly ) * ( \
    -1 * Kz + Lz ) + ( -1 * Iy + Jy ) * ( Jz + -1 * Kz ) * ( -1 * Iy + Ly \
    ) * ( -1 * Kz + Lz ) ) ) ) ) ) ) ) ) ) ) ) * ( -2 * ( ( -1 * Kx + Lx \
    ) )**( 2 ) * ( -1 * Iy + Ly ) + ( -2 * ( -1 * Iy + Ly ) * ( ( -1 * Kz \
    + Lz ) )**( 2 ) + -2 * ( -1 * ( -1 * Ix + Lx ) * ( -1 * Kx + Lx ) * ( \
    -1 * Ky + Ly ) + -1 * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) * ( -1 * Kz \
    + Lz ) ) ) ) * ( ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iy + Ly ) \
    )**( 2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * Ky + Ly ) )**( 2 \
    ) + ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( ( \
    ( -1 * Ky + Ly ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( ( ( -1 * \
    Ix + Lx ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + ( ( ( -1 * Iy + Ly \
    ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + -2 * ( ( -1 * Ix + Lx ) * \
    ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) + ( ( -1 * Ix \
    + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) * ( -1 * Kz + Lz ) + ( \
    -1 * Iy + Ly ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) * ( -1 * Kz + Lz \
    ) ) ) ) ) ) ) ) ) )**( -3/2 ) + ( ( ( ( ( -1 * Iy + Jy ) )**( 2 ) * ( \
    ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( -1 * Iz + Jz ) )**( 2 ) * ( ( Jx + \
    -1 * Kx ) )**( 2 ) + ( ( ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jy + -1 * Ky \
    ) )**( 2 ) + ( ( ( -1 * Iz + Jz ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( \
    2 ) + ( -2 * ( ( -1 * Ix + Jx ) * ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) \
    * ( Jy + -1 * Ky ) + ( ( -1 * Ix + Jx ) * ( -1 * Iz + Jz ) * ( Jx + \
    -1 * Kx ) * ( Jz + -1 * Kz ) + ( -1 * Iy + Jy ) * ( -1 * Iz + Jz ) * \
    ( Jy + -1 * Ky ) * ( Jz + -1 * Kz ) ) ) + ( ( ( -1 * Ix + Jx ) )**( 2 \
    ) * ( ( Jz + -1 * Kz ) )**( 2 ) + ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( \
    Jz + -1 * Kz ) )**( 2 ) ) ) ) ) ) ) )**( -1/2 ) * ( -1 * ( -1 * Iy + \
    Jy ) * ( Jx + -1 * Kx ) * ( -1 * Kx + Lx ) + ( ( -1 * Ix + Jx ) * ( \
    Jy + -1 * Ky ) * ( -1 * Kx + Lx ) + ( -1 * ( Jx + -1 * Kx ) * ( -1 * \
    Kx + Lx ) * ( -1 * Iy + Ly ) + ( ( Jx + -1 * Kx ) * ( -1 * Ix + Lx ) \
    * ( -1 * Ky + Ly ) + ( ( Jz + -1 * Kz ) * ( -1 * Ky + Ly ) * ( -1 * \
    Iz + Lz ) + ( ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( -1 * Kz + Lz ) \
    + ( -1 * ( -1 * Iy + Jy ) * ( Jz + -1 * Kz ) * ( -1 * Kz + Lz ) + -1 \
    * ( Jz + -1 * Kz ) * ( -1 * Iy + Ly ) * ( -1 * Kz + Lz ) ) ) ) ) ) ) \
    ) * ( ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iy + Ly ) )**( 2 ) + ( \
    ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * Ky + Ly ) )**( 2 ) + ( ( ( -1 \
    * Kx + Lx ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( ( ( -1 * Ky + \
    Ly ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( ( ( -1 * Ix + Lx ) \
    )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + ( ( ( -1 * Iy + Ly ) )**( 2 \
    ) * ( ( -1 * Kz + Lz ) )**( 2 ) + -2 * ( ( -1 * Ix + Lx ) * ( -1 * Kx \
    + Lx ) * ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) + ( ( -1 * Ix + Lx ) * ( \
    -1 * Kx + Lx ) * ( -1 * Iz + Lz ) * ( -1 * Kz + Lz ) + ( -1 * Iy + Ly \
    ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) * ( -1 * Kz + Lz ) ) ) ) ) ) \
    ) ) ) )**( -1/2 ) + -1/2 * ( -2 * ( -1 * Iy + Jy ) * ( ( Jx + -1 * Kx \
    ) )**( 2 ) + ( -2 * ( -1 * ( -1 * Ix + Jx ) * ( Jx + -1 * Kx ) * ( Jy \
    + -1 * Ky ) + -1 * ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( Jz + -1 * \
    Kz ) ) + -2 * ( -1 * Iy + Jy ) * ( ( Jz + -1 * Kz ) )**( 2 ) ) ) * ( \
    ( ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( \
    -1 * Iz + Jz ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( -1 * Ix \
    + Jx ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( ( ( -1 * Iz + Jz ) \
    )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( -2 * ( ( -1 * Ix + Jx ) * \
    ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( Jy + -1 * Ky ) + ( ( -1 * Ix \
    + Jx ) * ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( Jz + -1 * Kz ) + ( \
    -1 * Iy + Jy ) * ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( Jz + -1 * Kz \
    ) ) ) + ( ( ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) + \
    ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) ) ) ) ) ) ) \
    )**( -3/2 ) * ( ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( -1 * Kx + Lx \
    ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Ix + Jx ) * ( Jy + -1 * Ky ) * ( \
    -1 * Kx + Lx ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Iy + Jy ) * ( Jx + \
    -1 * Kx ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly ) + ( ( -1 * Ix + Jx ) \
    * ( Jy + -1 * Ky ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly ) + ( ( -1 * \
    Iz + Jz ) * ( Jx + -1 * Kx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) + \
    ( -1 * ( -1 * Ix + Jx ) * ( Jz + -1 * Kz ) * ( -1 * Kx + Lx ) * ( -1 \
    * Iz + Lz ) + ( ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( -1 * Ky + Ly \
    ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iy + Jy ) * ( Jz + -1 * Kz ) * ( \
    -1 * Ky + Ly ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iz + Jz ) * ( Jx + \
    -1 * Kx ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz ) + ( ( -1 * Ix + Jx ) \
    * ( Jz + -1 * Kz ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz ) + ( -1 * ( \
    -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( -1 * Iy + Ly ) * ( -1 * Kz + Lz \
    ) + ( -1 * Iy + Jy ) * ( Jz + -1 * Kz ) * ( -1 * Iy + Ly ) * ( -1 * \
    Kz + Lz ) ) ) ) ) ) ) ) ) ) ) ) * ( ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( \
    ( -1 * Iy + Ly ) )**( 2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * \
    Ky + Ly ) )**( 2 ) + ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iz + Lz \
    ) )**( 2 ) + ( ( ( -1 * Ky + Ly ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( \
    2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + ( \
    ( ( -1 * Iy + Ly ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + -2 * ( ( \
    -1 * Ix + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) * ( -1 * Ky + Ly \
    ) + ( ( -1 * Ix + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) * ( -1 * \
    Kz + Lz ) + ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) * \
    ( -1 * Kz + Lz ) ) ) ) ) ) ) ) ) )**( -1/2 ) ) )
                                                              
    ddIz = ( -1/2 * ( ( ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 \
    ) + ( ( ( -1 * Iz + Jz ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( \
    ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( ( ( -1 * \
    Iz + Jz ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( -2 * ( ( -1 * Ix \
    + Jx ) * ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( Jy + -1 * Ky ) + ( ( \
    -1 * Ix + Jx ) * ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( Jz + -1 * Kz \
    ) + ( -1 * Iy + Jy ) * ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( Jz + \
    -1 * Kz ) ) ) + ( ( ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jz + -1 * Kz ) \
    )**( 2 ) + ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) \
    ) ) ) ) ) ) )**( -1/2 ) * ( ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( \
    -1 * Kx + Lx ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Ix + Jx ) * ( Jy + \
    -1 * Ky ) * ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Iy + \
    Jy ) * ( Jx + -1 * Kx ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly ) + ( ( \
    -1 * Ix + Jx ) * ( Jy + -1 * Ky ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly \
    ) + ( ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( -1 * Kx + Lx ) * ( -1 * \
    Iz + Lz ) + ( -1 * ( -1 * Ix + Jx ) * ( Jz + -1 * Kz ) * ( -1 * Kx + \
    Lx ) * ( -1 * Iz + Lz ) + ( ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( \
    -1 * Ky + Ly ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iy + Jy ) * ( Jz + \
    -1 * Kz ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iz + \
    Jz ) * ( Jx + -1 * Kx ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz ) + ( ( \
    -1 * Ix + Jx ) * ( Jz + -1 * Kz ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz \
    ) + ( -1 * ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( -1 * Iy + Ly ) * ( \
    -1 * Kz + Lz ) + ( -1 * Iy + Jy ) * ( Jz + -1 * Kz ) * ( -1 * Iy + Ly \
    ) * ( -1 * Kz + Lz ) ) ) ) ) ) ) ) ) ) ) ) * ( -2 * ( ( -1 * Kx + Lx \
    ) )**( 2 ) * ( -1 * Iz + Lz ) + ( -2 * ( ( -1 * Ky + Ly ) )**( 2 ) * \
    ( -1 * Iz + Lz ) + -2 * ( -1 * ( -1 * Ix + Lx ) * ( -1 * Kx + Lx ) * \
    ( -1 * Kz + Lz ) + -1 * ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) * ( -1 * \
    Kz + Lz ) ) ) ) * ( ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iy + Ly \
    ) )**( 2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * Ky + Ly ) )**( \
    2 ) + ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( \
    ( ( -1 * Ky + Ly ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( ( ( -1 \
    * Ix + Lx ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + ( ( ( -1 * Iy + \
    Ly ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + -2 * ( ( -1 * Ix + Lx ) \
    * ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) + ( ( -1 * \
    Ix + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) * ( -1 * Kz + Lz ) + \
    ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) * ( -1 * Kz + \
    Lz ) ) ) ) ) ) ) ) ) )**( -3/2 ) + ( ( ( ( ( -1 * Iy + Jy ) )**( 2 ) \
    * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( -1 * Iz + Jz ) )**( 2 ) * ( ( \
    Jx + -1 * Kx ) )**( 2 ) + ( ( ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jy + -1 \
    * Ky ) )**( 2 ) + ( ( ( -1 * Iz + Jz ) )**( 2 ) * ( ( Jy + -1 * Ky ) \
    )**( 2 ) + ( -2 * ( ( -1 * Ix + Jx ) * ( -1 * Iy + Jy ) * ( Jx + -1 * \
    Kx ) * ( Jy + -1 * Ky ) + ( ( -1 * Ix + Jx ) * ( -1 * Iz + Jz ) * ( \
    Jx + -1 * Kx ) * ( Jz + -1 * Kz ) + ( -1 * Iy + Jy ) * ( -1 * Iz + Jz \
    ) * ( Jy + -1 * Ky ) * ( Jz + -1 * Kz ) ) ) + ( ( ( -1 * Ix + Jx ) \
    )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) + ( ( -1 * Iy + Jy ) )**( 2 ) \
    * ( ( Jz + -1 * Kz ) )**( 2 ) ) ) ) ) ) ) )**( -1/2 ) * ( -1 * ( -1 * \
    Iz + Jz ) * ( Jx + -1 * Kx ) * ( -1 * Kx + Lx ) + ( ( -1 * Ix + Jx ) \
    * ( Jz + -1 * Kz ) * ( -1 * Kx + Lx ) + ( -1 * ( -1 * Iz + Jz ) * ( \
    Jy + -1 * Ky ) * ( -1 * Ky + Ly ) + ( ( -1 * Iy + Jy ) * ( Jz + -1 * \
    Kz ) * ( -1 * Ky + Ly ) + ( -1 * ( Jx + -1 * Kx ) * ( -1 * Kx + Lx ) \
    * ( -1 * Iz + Lz ) + ( -1 * ( Jy + -1 * Ky ) * ( -1 * Ky + Ly ) * ( \
    -1 * Iz + Lz ) + ( ( Jx + -1 * Kx ) * ( -1 * Ix + Lx ) * ( -1 * Kz + \
    Lz ) + ( Jy + -1 * Ky ) * ( -1 * Iy + Ly ) * ( -1 * Kz + Lz ) ) ) ) ) \
    ) ) ) * ( ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iy + Ly ) )**( 2 ) \
    + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * Ky + Ly ) )**( 2 ) + ( ( ( \
    -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( ( ( -1 * Ky \
    + Ly ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( ( ( -1 * Ix + Lx ) \
    )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + ( ( ( -1 * Iy + Ly ) )**( 2 \
    ) * ( ( -1 * Kz + Lz ) )**( 2 ) + -2 * ( ( -1 * Ix + Lx ) * ( -1 * Kx \
    + Lx ) * ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) + ( ( -1 * Ix + Lx ) * ( \
    -1 * Kx + Lx ) * ( -1 * Iz + Lz ) * ( -1 * Kz + Lz ) + ( -1 * Iy + Ly \
    ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) * ( -1 * Kz + Lz ) ) ) ) ) ) \
    ) ) ) )**( -1/2 ) + -1/2 * ( -2 * ( -1 * Iz + Jz ) * ( ( Jx + -1 * Kx \
    ) )**( 2 ) + ( -2 * ( -1 * Iz + Jz ) * ( ( Jy + -1 * Ky ) )**( 2 ) + \
    -2 * ( -1 * ( -1 * Ix + Jx ) * ( Jx + -1 * Kx ) * ( Jz + -1 * Kz ) + \
    -1 * ( -1 * Iy + Jy ) * ( Jy + -1 * Ky ) * ( Jz + -1 * Kz ) ) ) ) * ( \
    ( ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( \
    -1 * Iz + Jz ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( -1 * Ix \
    + Jx ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( ( ( -1 * Iz + Jz ) \
    )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( -2 * ( ( -1 * Ix + Jx ) * \
    ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( Jy + -1 * Ky ) + ( ( -1 * Ix \
    + Jx ) * ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( Jz + -1 * Kz ) + ( \
    -1 * Iy + Jy ) * ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( Jz + -1 * Kz \
    ) ) ) + ( ( ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) + \
    ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) ) ) ) ) ) ) \
    )**( -3/2 ) * ( ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( -1 * Kx + Lx \
    ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Ix + Jx ) * ( Jy + -1 * Ky ) * ( \
    -1 * Kx + Lx ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Iy + Jy ) * ( Jx + \
    -1 * Kx ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly ) + ( ( -1 * Ix + Jx ) \
    * ( Jy + -1 * Ky ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly ) + ( ( -1 * \
    Iz + Jz ) * ( Jx + -1 * Kx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) + \
    ( -1 * ( -1 * Ix + Jx ) * ( Jz + -1 * Kz ) * ( -1 * Kx + Lx ) * ( -1 \
    * Iz + Lz ) + ( ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( -1 * Ky + Ly \
    ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iy + Jy ) * ( Jz + -1 * Kz ) * ( \
    -1 * Ky + Ly ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iz + Jz ) * ( Jx + \
    -1 * Kx ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz ) + ( ( -1 * Ix + Jx ) \
    * ( Jz + -1 * Kz ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz ) + ( -1 * ( \
    -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( -1 * Iy + Ly ) * ( -1 * Kz + Lz \
    ) + ( -1 * Iy + Jy ) * ( Jz + -1 * Kz ) * ( -1 * Iy + Ly ) * ( -1 * \
    Kz + Lz ) ) ) ) ) ) ) ) ) ) ) ) * ( ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( \
    ( -1 * Iy + Ly ) )**( 2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * \
    Ky + Ly ) )**( 2 ) + ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iz + Lz \
    ) )**( 2 ) + ( ( ( -1 * Ky + Ly ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( \
    2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + ( \
    ( ( -1 * Iy + Ly ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + -2 * ( ( \
    -1 * Ix + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) * ( -1 * Ky + Ly \
    ) + ( ( -1 * Ix + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) * ( -1 * \
    Kz + Lz ) + ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) * \
    ( -1 * Kz + Lz ) ) ) ) ) ) ) ) ) )**( -1/2 ) ) )
                                                                    
    ddJx = ( ( ( ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( \
    ( -1 * Iz + Jz ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( -1 * \
    Ix + Jx ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( ( ( -1 * Iz + Jz \
    ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( -2 * ( ( -1 * Ix + Jx ) \
    * ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( Jy + -1 * Ky ) + ( ( -1 * \
    Ix + Jx ) * ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( Jz + -1 * Kz ) + \
    ( -1 * Iy + Jy ) * ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( Jz + -1 * \
    Kz ) ) ) + ( ( ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 \
    ) + ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) ) ) ) ) \
    ) ) )**( -1/2 ) * ( ( -1 * Iy + Jy ) * ( -1 * Kx + Lx ) * ( -1 * Iy + \
    Ly ) + ( -1 * ( Jy + -1 * Ky ) * ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) \
    + ( -1 * ( -1 * Iy + Jy ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly ) + ( ( \
    Jy + -1 * Ky ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly ) + ( ( -1 * Iz + \
    Jz ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) + ( -1 * ( Jz + -1 * Kz ) \
    * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iz + Jz ) * ( \
    -1 * Ix + Lx ) * ( -1 * Kz + Lz ) + ( Jz + -1 * Kz ) * ( -1 * Ix + Lx \
    ) * ( -1 * Kz + Lz ) ) ) ) ) ) ) ) * ( ( ( ( -1 * Kx + Lx ) )**( 2 ) \
    * ( ( -1 * Iy + Ly ) )**( 2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( \
    -1 * Ky + Ly ) )**( 2 ) + ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iz \
    + Lz ) )**( 2 ) + ( ( ( -1 * Ky + Ly ) )**( 2 ) * ( ( -1 * Iz + Lz ) \
    )**( 2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 \
    ) + ( ( ( -1 * Iy + Ly ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + -2 \
    * ( ( -1 * Ix + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) * ( -1 * \
    Ky + Ly ) + ( ( -1 * Ix + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) \
    * ( -1 * Kz + Lz ) + ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) * ( -1 * Iz \
    + Lz ) * ( -1 * Kz + Lz ) ) ) ) ) ) ) ) ) )**( -1/2 ) + -1/2 * ( 2 * \
    ( ( -1 * Iy + Jy ) )**( 2 ) * ( Jx + -1 * Kx ) + ( 2 * ( ( -1 * Iz + \
    Jz ) )**( 2 ) * ( Jx + -1 * Kx ) + ( 2 * ( -1 * Ix + Jx ) * ( ( Jy + \
    -1 * Ky ) )**( 2 ) + ( -2 * ( ( -1 * Ix + Jx ) * ( -1 * Iy + Jy ) * ( \
    Jy + -1 * Ky ) + ( ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( Jy + -1 * \
    Ky ) + ( ( -1 * Ix + Jx ) * ( -1 * Iz + Jz ) * ( Jz + -1 * Kz ) + ( \
    -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( Jz + -1 * Kz ) ) ) ) + 2 * ( -1 \
    * Ix + Jx ) * ( ( Jz + -1 * Kz ) )**( 2 ) ) ) ) ) * ( ( ( ( -1 * Iy + \
    Jy ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( -1 * Iz + Jz ) \
    )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( -1 * Ix + Jx ) )**( 2 \
    ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( ( ( -1 * Iz + Jz ) )**( 2 ) * ( ( \
    Jy + -1 * Ky ) )**( 2 ) + ( -2 * ( ( -1 * Ix + Jx ) * ( -1 * Iy + Jy \
    ) * ( Jx + -1 * Kx ) * ( Jy + -1 * Ky ) + ( ( -1 * Ix + Jx ) * ( -1 * \
    Iz + Jz ) * ( Jx + -1 * Kx ) * ( Jz + -1 * Kz ) + ( -1 * Iy + Jy ) * \
    ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( Jz + -1 * Kz ) ) ) + ( ( ( -1 \
    * Ix + Jx ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) + ( ( -1 * Iy + Jy \
    ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) ) ) ) ) ) ) )**( -3/2 ) * ( \
    ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( -1 * Kx + Lx ) * ( -1 * Iy + \
    Ly ) + ( -1 * ( -1 * Ix + Jx ) * ( Jy + -1 * Ky ) * ( -1 * Kx + Lx ) \
    * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( \
    -1 * Ix + Lx ) * ( -1 * Ky + Ly ) + ( ( -1 * Ix + Jx ) * ( Jy + -1 * \
    Ky ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly ) + ( ( -1 * Iz + Jz ) * ( \
    Jx + -1 * Kx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * \
    Ix + Jx ) * ( Jz + -1 * Kz ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) + \
    ( ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( -1 * Ky + Ly ) * ( -1 * Iz \
    + Lz ) + ( -1 * ( -1 * Iy + Jy ) * ( Jz + -1 * Kz ) * ( -1 * Ky + Ly \
    ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( \
    -1 * Ix + Lx ) * ( -1 * Kz + Lz ) + ( ( -1 * Ix + Jx ) * ( Jz + -1 * \
    Kz ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz ) + ( -1 * ( -1 * Iz + Jz ) \
    * ( Jy + -1 * Ky ) * ( -1 * Iy + Ly ) * ( -1 * Kz + Lz ) + ( -1 * Iy \
    + Jy ) * ( Jz + -1 * Kz ) * ( -1 * Iy + Ly ) * ( -1 * Kz + Lz ) ) ) ) \
    ) ) ) ) ) ) ) ) * ( ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iy + Ly \
    ) )**( 2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * Ky + Ly ) )**( \
    2 ) + ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( \
    ( ( -1 * Ky + Ly ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( ( ( -1 \
    * Ix + Lx ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + ( ( ( -1 * Iy + \
    Ly ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + -2 * ( ( -1 * Ix + Lx ) \
    * ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) + ( ( -1 * \
    Ix + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) * ( -1 * Kz + Lz ) + \
    ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) * ( -1 * Kz + \
    Lz ) ) ) ) ) ) ) ) ) )**( -1/2 ) )
                                                        
    ddJy = ( ( ( ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( \
    ( -1 * Iz + Jz ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( -1 * \
    Ix + Jx ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( ( ( -1 * Iz + Jz \
    ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( -2 * ( ( -1 * Ix + Jx ) \
    * ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( Jy + -1 * Ky ) + ( ( -1 * \
    Ix + Jx ) * ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( Jz + -1 * Kz ) + \
    ( -1 * Iy + Jy ) * ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( Jz + -1 * \
    Kz ) ) ) + ( ( ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 \
    ) + ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) ) ) ) ) \
    ) ) )**( -1/2 ) * ( -1 * ( -1 * Ix + Jx ) * ( -1 * Kx + Lx ) * ( -1 * \
    Iy + Ly ) + ( ( Jx + -1 * Kx ) * ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) \
    + ( ( -1 * Ix + Jx ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly ) + ( -1 * ( \
    Jx + -1 * Kx ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly ) + ( ( -1 * Iz + \
    Jz ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) + ( -1 * ( Jz + -1 * Kz ) \
    * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iz + Jz ) * ( \
    -1 * Iy + Ly ) * ( -1 * Kz + Lz ) + ( Jz + -1 * Kz ) * ( -1 * Iy + Ly \
    ) * ( -1 * Kz + Lz ) ) ) ) ) ) ) ) * ( ( ( ( -1 * Kx + Lx ) )**( 2 ) \
    * ( ( -1 * Iy + Ly ) )**( 2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( \
    -1 * Ky + Ly ) )**( 2 ) + ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iz \
    + Lz ) )**( 2 ) + ( ( ( -1 * Ky + Ly ) )**( 2 ) * ( ( -1 * Iz + Lz ) \
    )**( 2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 \
    ) + ( ( ( -1 * Iy + Ly ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + -2 \
    * ( ( -1 * Ix + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) * ( -1 * \
    Ky + Ly ) + ( ( -1 * Ix + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) \
    * ( -1 * Kz + Lz ) + ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) * ( -1 * Iz \
    + Lz ) * ( -1 * Kz + Lz ) ) ) ) ) ) ) ) ) )**( -1/2 ) + -1/2 * ( 2 * \
    ( -1 * Iy + Jy ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( 2 * ( ( -1 * Ix + \
    Jx ) )**( 2 ) * ( Jy + -1 * Ky ) + ( 2 * ( ( -1 * Iz + Jz ) )**( 2 ) \
    * ( Jy + -1 * Ky ) + ( -2 * ( ( -1 * Ix + Jx ) * ( -1 * Iy + Jy ) * ( \
    Jx + -1 * Kx ) + ( ( -1 * Ix + Jx ) * ( Jx + -1 * Kx ) * ( Jy + -1 * \
    Ky ) + ( ( -1 * Iy + Jy ) * ( -1 * Iz + Jz ) * ( Jz + -1 * Kz ) + ( \
    -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( Jz + -1 * Kz ) ) ) ) + 2 * ( -1 \
    * Iy + Jy ) * ( ( Jz + -1 * Kz ) )**( 2 ) ) ) ) ) * ( ( ( ( -1 * Iy + \
    Jy ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( -1 * Iz + Jz ) \
    )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( -1 * Ix + Jx ) )**( 2 \
    ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( ( ( -1 * Iz + Jz ) )**( 2 ) * ( ( \
    Jy + -1 * Ky ) )**( 2 ) + ( -2 * ( ( -1 * Ix + Jx ) * ( -1 * Iy + Jy \
    ) * ( Jx + -1 * Kx ) * ( Jy + -1 * Ky ) + ( ( -1 * Ix + Jx ) * ( -1 * \
    Iz + Jz ) * ( Jx + -1 * Kx ) * ( Jz + -1 * Kz ) + ( -1 * Iy + Jy ) * \
    ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( Jz + -1 * Kz ) ) ) + ( ( ( -1 \
    * Ix + Jx ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) + ( ( -1 * Iy + Jy \
    ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) ) ) ) ) ) ) )**( -3/2 ) * ( \
    ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( -1 * Kx + Lx ) * ( -1 * Iy + \
    Ly ) + ( -1 * ( -1 * Ix + Jx ) * ( Jy + -1 * Ky ) * ( -1 * Kx + Lx ) \
    * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( \
    -1 * Ix + Lx ) * ( -1 * Ky + Ly ) + ( ( -1 * Ix + Jx ) * ( Jy + -1 * \
    Ky ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly ) + ( ( -1 * Iz + Jz ) * ( \
    Jx + -1 * Kx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * \
    Ix + Jx ) * ( Jz + -1 * Kz ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) + \
    ( ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( -1 * Ky + Ly ) * ( -1 * Iz \
    + Lz ) + ( -1 * ( -1 * Iy + Jy ) * ( Jz + -1 * Kz ) * ( -1 * Ky + Ly \
    ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( \
    -1 * Ix + Lx ) * ( -1 * Kz + Lz ) + ( ( -1 * Ix + Jx ) * ( Jz + -1 * \
    Kz ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz ) + ( -1 * ( -1 * Iz + Jz ) \
    * ( Jy + -1 * Ky ) * ( -1 * Iy + Ly ) * ( -1 * Kz + Lz ) + ( -1 * Iy \
    + Jy ) * ( Jz + -1 * Kz ) * ( -1 * Iy + Ly ) * ( -1 * Kz + Lz ) ) ) ) \
    ) ) ) ) ) ) ) ) * ( ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iy + Ly \
    ) )**( 2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * Ky + Ly ) )**( \
    2 ) + ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( \
    ( ( -1 * Ky + Ly ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( ( ( -1 \
    * Ix + Lx ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + ( ( ( -1 * Iy + \
    Ly ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + -2 * ( ( -1 * Ix + Lx ) \
    * ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) + ( ( -1 * \
    Ix + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) * ( -1 * Kz + Lz ) + \
    ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) * ( -1 * Kz + \
    Lz ) ) ) ) ) ) ) ) ) )**( -1/2 ) )                                                    
                 
    ddJz = ( ( ( ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( \
    ( -1 * Iz + Jz ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( -1 * \
    Ix + Jx ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( ( ( -1 * Iz + Jz \
    ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( -2 * ( ( -1 * Ix + Jx ) \
    * ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( Jy + -1 * Ky ) + ( ( -1 * \
    Ix + Jx ) * ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( Jz + -1 * Kz ) + \
    ( -1 * Iy + Jy ) * ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( Jz + -1 * \
    Kz ) ) ) + ( ( ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 \
    ) + ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) ) ) ) ) \
    ) ) )**( -1/2 ) * ( -1 * ( -1 * Ix + Jx ) * ( -1 * Kx + Lx ) * ( -1 * \
    Iz + Lz ) + ( ( Jx + -1 * Kx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) \
    + ( -1 * ( -1 * Iy + Jy ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) + ( ( \
    Jy + -1 * Ky ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) + ( ( -1 * Ix + \
    Jx ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz ) + ( -1 * ( Jx + -1 * Kx ) \
    * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz ) + ( ( -1 * Iy + Jy ) * ( -1 * \
    Iy + Ly ) * ( -1 * Kz + Lz ) + -1 * ( Jy + -1 * Ky ) * ( -1 * Iy + Ly \
    ) * ( -1 * Kz + Lz ) ) ) ) ) ) ) ) * ( ( ( ( -1 * Kx + Lx ) )**( 2 ) \
    * ( ( -1 * Iy + Ly ) )**( 2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( \
    -1 * Ky + Ly ) )**( 2 ) + ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iz \
    + Lz ) )**( 2 ) + ( ( ( -1 * Ky + Ly ) )**( 2 ) * ( ( -1 * Iz + Lz ) \
    )**( 2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 \
    ) + ( ( ( -1 * Iy + Ly ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + -2 \
    * ( ( -1 * Ix + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) * ( -1 * \
    Ky + Ly ) + ( ( -1 * Ix + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) \
    * ( -1 * Kz + Lz ) + ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) * ( -1 * Iz \
    + Lz ) * ( -1 * Kz + Lz ) ) ) ) ) ) ) ) ) )**( -1/2 ) + -1/2 * ( 2 * \
    ( -1 * Iz + Jz ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( 2 * ( -1 * Iz + Jz \
    ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( -2 * ( ( -1 * Ix + Jx ) * ( -1 * \
    Iz + Jz ) * ( Jx + -1 * Kx ) + ( ( -1 * Iy + Jy ) * ( -1 * Iz + Jz ) \
    * ( Jy + -1 * Ky ) + ( ( -1 * Ix + Jx ) * ( Jx + -1 * Kx ) * ( Jz + \
    -1 * Kz ) + ( -1 * Iy + Jy ) * ( Jy + -1 * Ky ) * ( Jz + -1 * Kz ) ) \
    ) ) + ( 2 * ( ( -1 * Ix + Jx ) )**( 2 ) * ( Jz + -1 * Kz ) + 2 * ( ( \
    -1 * Iy + Jy ) )**( 2 ) * ( Jz + -1 * Kz ) ) ) ) ) * ( ( ( ( -1 * Iy \
    + Jy ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( -1 * Iz + Jz ) \
    )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( -1 * Ix + Jx ) )**( 2 \
    ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( ( ( -1 * Iz + Jz ) )**( 2 ) * ( ( \
    Jy + -1 * Ky ) )**( 2 ) + ( -2 * ( ( -1 * Ix + Jx ) * ( -1 * Iy + Jy \
    ) * ( Jx + -1 * Kx ) * ( Jy + -1 * Ky ) + ( ( -1 * Ix + Jx ) * ( -1 * \
    Iz + Jz ) * ( Jx + -1 * Kx ) * ( Jz + -1 * Kz ) + ( -1 * Iy + Jy ) * \
    ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( Jz + -1 * Kz ) ) ) + ( ( ( -1 \
    * Ix + Jx ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) + ( ( -1 * Iy + Jy \
    ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) ) ) ) ) ) ) )**( -3/2 ) * ( \
    ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( -1 * Kx + Lx ) * ( -1 * Iy + \
    Ly ) + ( -1 * ( -1 * Ix + Jx ) * ( Jy + -1 * Ky ) * ( -1 * Kx + Lx ) \
    * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( \
    -1 * Ix + Lx ) * ( -1 * Ky + Ly ) + ( ( -1 * Ix + Jx ) * ( Jy + -1 * \
    Ky ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly ) + ( ( -1 * Iz + Jz ) * ( \
    Jx + -1 * Kx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * \
    Ix + Jx ) * ( Jz + -1 * Kz ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) + \
    ( ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( -1 * Ky + Ly ) * ( -1 * Iz \
    + Lz ) + ( -1 * ( -1 * Iy + Jy ) * ( Jz + -1 * Kz ) * ( -1 * Ky + Ly \
    ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( \
    -1 * Ix + Lx ) * ( -1 * Kz + Lz ) + ( ( -1 * Ix + Jx ) * ( Jz + -1 * \
    Kz ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz ) + ( -1 * ( -1 * Iz + Jz ) \
    * ( Jy + -1 * Ky ) * ( -1 * Iy + Ly ) * ( -1 * Kz + Lz ) + ( -1 * Iy \
    + Jy ) * ( Jz + -1 * Kz ) * ( -1 * Iy + Ly ) * ( -1 * Kz + Lz ) ) ) ) \
    ) ) ) ) ) ) ) ) * ( ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iy + Ly \
    ) )**( 2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * Ky + Ly ) )**( \
    2 ) + ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( \
    ( ( -1 * Ky + Ly ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( ( ( -1 \
    * Ix + Lx ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + ( ( ( -1 * Iy + \
    Ly ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + -2 * ( ( -1 * Ix + Lx ) \
    * ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) + ( ( -1 * \
    Ix + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) * ( -1 * Kz + Lz ) + \
    ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) * ( -1 * Kz + \
    Lz ) ) ) ) ) ) ) ) ) )**( -1/2 ) )                                             
                      
                                                        
    ddKx = ( -1/2 * ( ( ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 \
    ) + ( ( ( -1 * Iz + Jz ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( \
    ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( ( ( -1 * \
    Iz + Jz ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( -2 * ( ( -1 * Ix \
    + Jx ) * ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( Jy + -1 * Ky ) + ( ( \
    -1 * Ix + Jx ) * ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( Jz + -1 * Kz \
    ) + ( -1 * Iy + Jy ) * ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( Jz + \
    -1 * Kz ) ) ) + ( ( ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jz + -1 * Kz ) \
    )**( 2 ) + ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) \
    ) ) ) ) ) ) )**( -1/2 ) * ( ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( \
    -1 * Kx + Lx ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Ix + Jx ) * ( Jy + \
    -1 * Ky ) * ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Iy + \
    Jy ) * ( Jx + -1 * Kx ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly ) + ( ( \
    -1 * Ix + Jx ) * ( Jy + -1 * Ky ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly \
    ) + ( ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( -1 * Kx + Lx ) * ( -1 * \
    Iz + Lz ) + ( -1 * ( -1 * Ix + Jx ) * ( Jz + -1 * Kz ) * ( -1 * Kx + \
    Lx ) * ( -1 * Iz + Lz ) + ( ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( \
    -1 * Ky + Ly ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iy + Jy ) * ( Jz + \
    -1 * Kz ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iz + \
    Jz ) * ( Jx + -1 * Kx ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz ) + ( ( \
    -1 * Ix + Jx ) * ( Jz + -1 * Kz ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz \
    ) + ( -1 * ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( -1 * Iy + Ly ) * ( \
    -1 * Kz + Lz ) + ( -1 * Iy + Jy ) * ( Jz + -1 * Kz ) * ( -1 * Iy + Ly \
    ) * ( -1 * Kz + Lz ) ) ) ) ) ) ) ) ) ) ) ) * ( -2 * ( -1 * Kx + Lx ) \
    * ( ( -1 * Iy + Ly ) )**( 2 ) + ( -2 * ( -1 * Kx + Lx ) * ( ( -1 * Iz \
    + Lz ) )**( 2 ) + -2 * ( -1 * ( -1 * Ix + Lx ) * ( -1 * Iy + Ly ) * ( \
    -1 * Ky + Ly ) + -1 * ( -1 * Ix + Lx ) * ( -1 * Iz + Lz ) * ( -1 * Kz \
    + Lz ) ) ) ) * ( ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iy + Ly ) \
    )**( 2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * Ky + Ly ) )**( 2 \
    ) + ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( ( \
    ( -1 * Ky + Ly ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( ( ( -1 * \
    Ix + Lx ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + ( ( ( -1 * Iy + Ly \
    ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + -2 * ( ( -1 * Ix + Lx ) * \
    ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) + ( ( -1 * Ix \
    + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) * ( -1 * Kz + Lz ) + ( \
    -1 * Iy + Ly ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) * ( -1 * Kz + Lz \
    ) ) ) ) ) ) ) ) ) )**( -3/2 ) + ( ( ( ( ( -1 * Iy + Jy ) )**( 2 ) * ( \
    ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( -1 * Iz + Jz ) )**( 2 ) * ( ( Jx + \
    -1 * Kx ) )**( 2 ) + ( ( ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jy + -1 * Ky \
    ) )**( 2 ) + ( ( ( -1 * Iz + Jz ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( \
    2 ) + ( -2 * ( ( -1 * Ix + Jx ) * ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) \
    * ( Jy + -1 * Ky ) + ( ( -1 * Ix + Jx ) * ( -1 * Iz + Jz ) * ( Jx + \
    -1 * Kx ) * ( Jz + -1 * Kz ) + ( -1 * Iy + Jy ) * ( -1 * Iz + Jz ) * \
    ( Jy + -1 * Ky ) * ( Jz + -1 * Kz ) ) ) + ( ( ( -1 * Ix + Jx ) )**( 2 \
    ) * ( ( Jz + -1 * Kz ) )**( 2 ) + ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( \
    Jz + -1 * Kz ) )**( 2 ) ) ) ) ) ) ) )**( -1/2 ) * ( -1 * ( -1 * Iy + \
    Jy ) * ( Jx + -1 * Kx ) * ( -1 * Iy + Ly ) + ( ( -1 * Ix + Jx ) * ( \
    Jy + -1 * Ky ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Iy + Jy ) * ( -1 * \
    Kx + Lx ) * ( -1 * Iy + Ly ) + ( ( -1 * Iy + Jy ) * ( -1 * Ix + Lx ) \
    * ( -1 * Ky + Ly ) + ( -1 * ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( \
    -1 * Iz + Lz ) + ( ( -1 * Ix + Jx ) * ( Jz + -1 * Kz ) * ( -1 * Iz + \
    Lz ) + ( -1 * ( -1 * Iz + Jz ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) \
    + ( -1 * Iz + Jz ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz ) ) ) ) ) ) ) \
    ) * ( ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iy + Ly ) )**( 2 ) + ( \
    ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * Ky + Ly ) )**( 2 ) + ( ( ( -1 \
    * Kx + Lx ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( ( ( -1 * Ky + \
    Ly ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( ( ( -1 * Ix + Lx ) \
    )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + ( ( ( -1 * Iy + Ly ) )**( 2 \
    ) * ( ( -1 * Kz + Lz ) )**( 2 ) + -2 * ( ( -1 * Ix + Lx ) * ( -1 * Kx \
    + Lx ) * ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) + ( ( -1 * Ix + Lx ) * ( \
    -1 * Kx + Lx ) * ( -1 * Iz + Lz ) * ( -1 * Kz + Lz ) + ( -1 * Iy + Ly \
    ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) * ( -1 * Kz + Lz ) ) ) ) ) ) \
    ) ) ) )**( -1/2 ) + -1/2 * ( -2 * ( ( -1 * Iy + Jy ) )**( 2 ) * ( Jx \
    + -1 * Kx ) + ( -2 * ( ( -1 * Iz + Jz ) )**( 2 ) * ( Jx + -1 * Kx ) + \
    -2 * ( -1 * ( -1 * Ix + Jx ) * ( -1 * Iy + Jy ) * ( Jy + -1 * Ky ) + \
    -1 * ( -1 * Ix + Jx ) * ( -1 * Iz + Jz ) * ( Jz + -1 * Kz ) ) ) ) * ( \
    ( ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( \
    -1 * Iz + Jz ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( -1 * Ix \
    + Jx ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( ( ( -1 * Iz + Jz ) \
    )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( -2 * ( ( -1 * Ix + Jx ) * \
    ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( Jy + -1 * Ky ) + ( ( -1 * Ix \
    + Jx ) * ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( Jz + -1 * Kz ) + ( \
    -1 * Iy + Jy ) * ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( Jz + -1 * Kz \
    ) ) ) + ( ( ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) + \
    ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) ) ) ) ) ) ) \
    )**( -3/2 ) * ( ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( -1 * Kx + Lx \
    ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Ix + Jx ) * ( Jy + -1 * Ky ) * ( \
    -1 * Kx + Lx ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Iy + Jy ) * ( Jx + \
    -1 * Kx ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly ) + ( ( -1 * Ix + Jx ) \
    * ( Jy + -1 * Ky ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly ) + ( ( -1 * \
    Iz + Jz ) * ( Jx + -1 * Kx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) + \
    ( -1 * ( -1 * Ix + Jx ) * ( Jz + -1 * Kz ) * ( -1 * Kx + Lx ) * ( -1 \
    * Iz + Lz ) + ( ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( -1 * Ky + Ly \
    ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iy + Jy ) * ( Jz + -1 * Kz ) * ( \
    -1 * Ky + Ly ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iz + Jz ) * ( Jx + \
    -1 * Kx ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz ) + ( ( -1 * Ix + Jx ) \
    * ( Jz + -1 * Kz ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz ) + ( -1 * ( \
    -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( -1 * Iy + Ly ) * ( -1 * Kz + Lz \
    ) + ( -1 * Iy + Jy ) * ( Jz + -1 * Kz ) * ( -1 * Iy + Ly ) * ( -1 * \
    Kz + Lz ) ) ) ) ) ) ) ) ) ) ) ) * ( ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( \
    ( -1 * Iy + Ly ) )**( 2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * \
    Ky + Ly ) )**( 2 ) + ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iz + Lz \
    ) )**( 2 ) + ( ( ( -1 * Ky + Ly ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( \
    2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + ( \
    ( ( -1 * Iy + Ly ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + -2 * ( ( \
    -1 * Ix + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) * ( -1 * Ky + Ly \
    ) + ( ( -1 * Ix + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) * ( -1 * \
    Kz + Lz ) + ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) * \
    ( -1 * Kz + Lz ) ) ) ) ) ) ) ) ) )**( -1/2 ) ) )   
                                                                    
    ddKy = ( -1/2 * ( ( ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 \
    ) + ( ( ( -1 * Iz + Jz ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( \
    ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( ( ( -1 * \
    Iz + Jz ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( -2 * ( ( -1 * Ix \
    + Jx ) * ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( Jy + -1 * Ky ) + ( ( \
    -1 * Ix + Jx ) * ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( Jz + -1 * Kz \
    ) + ( -1 * Iy + Jy ) * ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( Jz + \
    -1 * Kz ) ) ) + ( ( ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jz + -1 * Kz ) \
    )**( 2 ) + ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) \
    ) ) ) ) ) ) )**( -1/2 ) * ( ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( \
    -1 * Kx + Lx ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Ix + Jx ) * ( Jy + \
    -1 * Ky ) * ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Iy + \
    Jy ) * ( Jx + -1 * Kx ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly ) + ( ( \
    -1 * Ix + Jx ) * ( Jy + -1 * Ky ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly \
    ) + ( ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( -1 * Kx + Lx ) * ( -1 * \
    Iz + Lz ) + ( -1 * ( -1 * Ix + Jx ) * ( Jz + -1 * Kz ) * ( -1 * Kx + \
    Lx ) * ( -1 * Iz + Lz ) + ( ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( \
    -1 * Ky + Ly ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iy + Jy ) * ( Jz + \
    -1 * Kz ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iz + \
    Jz ) * ( Jx + -1 * Kx ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz ) + ( ( \
    -1 * Ix + Jx ) * ( Jz + -1 * Kz ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz \
    ) + ( -1 * ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( -1 * Iy + Ly ) * ( \
    -1 * Kz + Lz ) + ( -1 * Iy + Jy ) * ( Jz + -1 * Kz ) * ( -1 * Iy + Ly \
    ) * ( -1 * Kz + Lz ) ) ) ) ) ) ) ) ) ) ) ) * ( -2 * ( ( -1 * Ix + Lx \
    ) )**( 2 ) * ( -1 * Ky + Ly ) + ( -2 * ( -1 * Ky + Ly ) * ( ( -1 * Iz \
    + Lz ) )**( 2 ) + -2 * ( -1 * ( -1 * Ix + Lx ) * ( -1 * Kx + Lx ) * ( \
    -1 * Iy + Ly ) + -1 * ( -1 * Iy + Ly ) * ( -1 * Iz + Lz ) * ( -1 * Kz \
    + Lz ) ) ) ) * ( ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iy + Ly ) \
    )**( 2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * Ky + Ly ) )**( 2 \
    ) + ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( ( \
    ( -1 * Ky + Ly ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( ( ( -1 * \
    Ix + Lx ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + ( ( ( -1 * Iy + Ly \
    ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + -2 * ( ( -1 * Ix + Lx ) * \
    ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) + ( ( -1 * Ix \
    + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) * ( -1 * Kz + Lz ) + ( \
    -1 * Iy + Ly ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) * ( -1 * Kz + Lz \
    ) ) ) ) ) ) ) ) ) )**( -3/2 ) + ( ( ( ( ( -1 * Iy + Jy ) )**( 2 ) * ( \
    ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( -1 * Iz + Jz ) )**( 2 ) * ( ( Jx + \
    -1 * Kx ) )**( 2 ) + ( ( ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jy + -1 * Ky \
    ) )**( 2 ) + ( ( ( -1 * Iz + Jz ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( \
    2 ) + ( -2 * ( ( -1 * Ix + Jx ) * ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) \
    * ( Jy + -1 * Ky ) + ( ( -1 * Ix + Jx ) * ( -1 * Iz + Jz ) * ( Jx + \
    -1 * Kx ) * ( Jz + -1 * Kz ) + ( -1 * Iy + Jy ) * ( -1 * Iz + Jz ) * \
    ( Jy + -1 * Ky ) * ( Jz + -1 * Kz ) ) ) + ( ( ( -1 * Ix + Jx ) )**( 2 \
    ) * ( ( Jz + -1 * Kz ) )**( 2 ) + ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( \
    Jz + -1 * Kz ) )**( 2 ) ) ) ) ) ) ) )**( -1/2 ) * ( ( -1 * Iy + Jy ) \
    * ( Jx + -1 * Kx ) * ( -1 * Ix + Lx ) + ( -1 * ( -1 * Ix + Jx ) * ( \
    Jy + -1 * Ky ) * ( -1 * Ix + Lx ) + ( ( -1 * Ix + Jx ) * ( -1 * Kx + \
    Lx ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Ix + Jx ) * ( -1 * Ix + Lx ) \
    * ( -1 * Ky + Ly ) + ( -1 * ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( \
    -1 * Iz + Lz ) + ( ( -1 * Iy + Jy ) * ( Jz + -1 * Kz ) * ( -1 * Iz + \
    Lz ) + ( -1 * ( -1 * Iz + Jz ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) \
    + ( -1 * Iz + Jz ) * ( -1 * Iy + Ly ) * ( -1 * Kz + Lz ) ) ) ) ) ) ) \
    ) * ( ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iy + Ly ) )**( 2 ) + ( \
    ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * Ky + Ly ) )**( 2 ) + ( ( ( -1 \
    * Kx + Lx ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( ( ( -1 * Ky + \
    Ly ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( ( ( -1 * Ix + Lx ) \
    )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + ( ( ( -1 * Iy + Ly ) )**( 2 \
    ) * ( ( -1 * Kz + Lz ) )**( 2 ) + -2 * ( ( -1 * Ix + Lx ) * ( -1 * Kx \
    + Lx ) * ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) + ( ( -1 * Ix + Lx ) * ( \
    -1 * Kx + Lx ) * ( -1 * Iz + Lz ) * ( -1 * Kz + Lz ) + ( -1 * Iy + Ly \
    ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) * ( -1 * Kz + Lz ) ) ) ) ) ) \
    ) ) ) )**( -1/2 ) + -1/2 * ( -2 * ( ( -1 * Ix + Jx ) )**( 2 ) * ( Jy \
    + -1 * Ky ) + ( -2 * ( ( -1 * Iz + Jz ) )**( 2 ) * ( Jy + -1 * Ky ) + \
    -2 * ( -1 * ( -1 * Ix + Jx ) * ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) + \
    -1 * ( -1 * Iy + Jy ) * ( -1 * Iz + Jz ) * ( Jz + -1 * Kz ) ) ) ) * ( \
    ( ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( \
    -1 * Iz + Jz ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( -1 * Ix \
    + Jx ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( ( ( -1 * Iz + Jz ) \
    )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( -2 * ( ( -1 * Ix + Jx ) * \
    ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( Jy + -1 * Ky ) + ( ( -1 * Ix \
    + Jx ) * ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( Jz + -1 * Kz ) + ( \
    -1 * Iy + Jy ) * ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( Jz + -1 * Kz \
    ) ) ) + ( ( ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) + \
    ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) ) ) ) ) ) ) \
    )**( -3/2 ) * ( ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( -1 * Kx + Lx \
    ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Ix + Jx ) * ( Jy + -1 * Ky ) * ( \
    -1 * Kx + Lx ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Iy + Jy ) * ( Jx + \
    -1 * Kx ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly ) + ( ( -1 * Ix + Jx ) \
    * ( Jy + -1 * Ky ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly ) + ( ( -1 * \
    Iz + Jz ) * ( Jx + -1 * Kx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) + \
    ( -1 * ( -1 * Ix + Jx ) * ( Jz + -1 * Kz ) * ( -1 * Kx + Lx ) * ( -1 \
    * Iz + Lz ) + ( ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( -1 * Ky + Ly \
    ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iy + Jy ) * ( Jz + -1 * Kz ) * ( \
    -1 * Ky + Ly ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iz + Jz ) * ( Jx + \
    -1 * Kx ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz ) + ( ( -1 * Ix + Jx ) \
    * ( Jz + -1 * Kz ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz ) + ( -1 * ( \
    -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( -1 * Iy + Ly ) * ( -1 * Kz + Lz \
    ) + ( -1 * Iy + Jy ) * ( Jz + -1 * Kz ) * ( -1 * Iy + Ly ) * ( -1 * \
    Kz + Lz ) ) ) ) ) ) ) ) ) ) ) ) * ( ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( \
    ( -1 * Iy + Ly ) )**( 2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * \
    Ky + Ly ) )**( 2 ) + ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iz + Lz \
    ) )**( 2 ) + ( ( ( -1 * Ky + Ly ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( \
    2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + ( \
    ( ( -1 * Iy + Ly ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + -2 * ( ( \
    -1 * Ix + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) * ( -1 * Ky + Ly \
    ) + ( ( -1 * Ix + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) * ( -1 * \
    Kz + Lz ) + ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) * \
    ( -1 * Kz + Lz ) ) ) ) ) ) ) ) ) )**( -1/2 ) ) )                                                                
                                                                    
    ddKz = ( -1/2 * ( ( ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 \
    ) + ( ( ( -1 * Iz + Jz ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( \
    ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( ( ( -1 * \
    Iz + Jz ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( -2 * ( ( -1 * Ix \
    + Jx ) * ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( Jy + -1 * Ky ) + ( ( \
    -1 * Ix + Jx ) * ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( Jz + -1 * Kz \
    ) + ( -1 * Iy + Jy ) * ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( Jz + \
    -1 * Kz ) ) ) + ( ( ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jz + -1 * Kz ) \
    )**( 2 ) + ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) \
    ) ) ) ) ) ) )**( -1/2 ) * ( ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( \
    -1 * Kx + Lx ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Ix + Jx ) * ( Jy + \
    -1 * Ky ) * ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Iy + \
    Jy ) * ( Jx + -1 * Kx ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly ) + ( ( \
    -1 * Ix + Jx ) * ( Jy + -1 * Ky ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly \
    ) + ( ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( -1 * Kx + Lx ) * ( -1 * \
    Iz + Lz ) + ( -1 * ( -1 * Ix + Jx ) * ( Jz + -1 * Kz ) * ( -1 * Kx + \
    Lx ) * ( -1 * Iz + Lz ) + ( ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( \
    -1 * Ky + Ly ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iy + Jy ) * ( Jz + \
    -1 * Kz ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iz + \
    Jz ) * ( Jx + -1 * Kx ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz ) + ( ( \
    -1 * Ix + Jx ) * ( Jz + -1 * Kz ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz \
    ) + ( -1 * ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( -1 * Iy + Ly ) * ( \
    -1 * Kz + Lz ) + ( -1 * Iy + Jy ) * ( Jz + -1 * Kz ) * ( -1 * Iy + Ly \
    ) * ( -1 * Kz + Lz ) ) ) ) ) ) ) ) ) ) ) ) * ( -2 * ( ( -1 * Ix + Lx \
    ) )**( 2 ) * ( -1 * Kz + Lz ) + ( -2 * ( ( -1 * Iy + Ly ) )**( 2 ) * \
    ( -1 * Kz + Lz ) + -2 * ( -1 * ( -1 * Ix + Lx ) * ( -1 * Kx + Lx ) * \
    ( -1 * Iz + Lz ) + -1 * ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) * ( -1 * \
    Iz + Lz ) ) ) ) * ( ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iy + Ly \
    ) )**( 2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * Ky + Ly ) )**( \
    2 ) + ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( \
    ( ( -1 * Ky + Ly ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( ( ( -1 \
    * Ix + Lx ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + ( ( ( -1 * Iy + \
    Ly ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + -2 * ( ( -1 * Ix + Lx ) \
    * ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) + ( ( -1 * \
    Ix + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) * ( -1 * Kz + Lz ) + \
    ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) * ( -1 * Kz + \
    Lz ) ) ) ) ) ) ) ) ) )**( -3/2 ) + ( ( ( ( ( -1 * Iy + Jy ) )**( 2 ) \
    * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( -1 * Iz + Jz ) )**( 2 ) * ( ( \
    Jx + -1 * Kx ) )**( 2 ) + ( ( ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jy + -1 \
    * Ky ) )**( 2 ) + ( ( ( -1 * Iz + Jz ) )**( 2 ) * ( ( Jy + -1 * Ky ) \
    )**( 2 ) + ( -2 * ( ( -1 * Ix + Jx ) * ( -1 * Iy + Jy ) * ( Jx + -1 * \
    Kx ) * ( Jy + -1 * Ky ) + ( ( -1 * Ix + Jx ) * ( -1 * Iz + Jz ) * ( \
    Jx + -1 * Kx ) * ( Jz + -1 * Kz ) + ( -1 * Iy + Jy ) * ( -1 * Iz + Jz \
    ) * ( Jy + -1 * Ky ) * ( Jz + -1 * Kz ) ) ) + ( ( ( -1 * Ix + Jx ) \
    )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) + ( ( -1 * Iy + Jy ) )**( 2 ) \
    * ( ( Jz + -1 * Kz ) )**( 2 ) ) ) ) ) ) ) )**( -1/2 ) * ( ( -1 * Iz + \
    Jz ) * ( Jx + -1 * Kx ) * ( -1 * Ix + Lx ) + ( -1 * ( -1 * Ix + Jx ) \
    * ( Jz + -1 * Kz ) * ( -1 * Ix + Lx ) + ( ( -1 * Iz + Jz ) * ( Jy + \
    -1 * Ky ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Iy + Jy ) * ( Jz + -1 * \
    Kz ) * ( -1 * Iy + Ly ) + ( ( -1 * Ix + Jx ) * ( -1 * Kx + Lx ) * ( \
    -1 * Iz + Lz ) + ( ( -1 * Iy + Jy ) * ( -1 * Ky + Ly ) * ( -1 * Iz + \
    Lz ) + ( -1 * ( -1 * Ix + Jx ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz ) \
    + -1 * ( -1 * Iy + Jy ) * ( -1 * Iy + Ly ) * ( -1 * Kz + Lz ) ) ) ) ) \
    ) ) ) * ( ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iy + Ly ) )**( 2 ) \
    + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * Ky + Ly ) )**( 2 ) + ( ( ( \
    -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( ( ( -1 * Ky \
    + Ly ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( ( ( -1 * Ix + Lx ) \
    )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + ( ( ( -1 * Iy + Ly ) )**( 2 \
    ) * ( ( -1 * Kz + Lz ) )**( 2 ) + -2 * ( ( -1 * Ix + Lx ) * ( -1 * Kx \
    + Lx ) * ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) + ( ( -1 * Ix + Lx ) * ( \
    -1 * Kx + Lx ) * ( -1 * Iz + Lz ) * ( -1 * Kz + Lz ) + ( -1 * Iy + Ly \
    ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) * ( -1 * Kz + Lz ) ) ) ) ) ) \
    ) ) ) )**( -1/2 ) + -1/2 * ( -2 * ( -1 * ( -1 * Ix + Jx ) * ( -1 * Iz \
    + Jz ) * ( Jx + -1 * Kx ) + -1 * ( -1 * Iy + Jy ) * ( -1 * Iz + Jz ) \
    * ( Jy + -1 * Ky ) ) + ( -2 * ( ( -1 * Ix + Jx ) )**( 2 ) * ( Jz + -1 \
    * Kz ) + -2 * ( ( -1 * Iy + Jy ) )**( 2 ) * ( Jz + -1 * Kz ) ) ) * ( \
    ( ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( \
    -1 * Iz + Jz ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( -1 * Ix \
    + Jx ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( ( ( -1 * Iz + Jz ) \
    )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( -2 * ( ( -1 * Ix + Jx ) * \
    ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( Jy + -1 * Ky ) + ( ( -1 * Ix \
    + Jx ) * ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( Jz + -1 * Kz ) + ( \
    -1 * Iy + Jy ) * ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( Jz + -1 * Kz \
    ) ) ) + ( ( ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) + \
    ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) ) ) ) ) ) ) \
    )**( -3/2 ) * ( ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( -1 * Kx + Lx \
    ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Ix + Jx ) * ( Jy + -1 * Ky ) * ( \
    -1 * Kx + Lx ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Iy + Jy ) * ( Jx + \
    -1 * Kx ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly ) + ( ( -1 * Ix + Jx ) \
    * ( Jy + -1 * Ky ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly ) + ( ( -1 * \
    Iz + Jz ) * ( Jx + -1 * Kx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) + \
    ( -1 * ( -1 * Ix + Jx ) * ( Jz + -1 * Kz ) * ( -1 * Kx + Lx ) * ( -1 \
    * Iz + Lz ) + ( ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( -1 * Ky + Ly \
    ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iy + Jy ) * ( Jz + -1 * Kz ) * ( \
    -1 * Ky + Ly ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iz + Jz ) * ( Jx + \
    -1 * Kx ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz ) + ( ( -1 * Ix + Jx ) \
    * ( Jz + -1 * Kz ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz ) + ( -1 * ( \
    -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( -1 * Iy + Ly ) * ( -1 * Kz + Lz \
    ) + ( -1 * Iy + Jy ) * ( Jz + -1 * Kz ) * ( -1 * Iy + Ly ) * ( -1 * \
    Kz + Lz ) ) ) ) ) ) ) ) ) ) ) ) * ( ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( \
    ( -1 * Iy + Ly ) )**( 2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * \
    Ky + Ly ) )**( 2 ) + ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iz + Lz \
    ) )**( 2 ) + ( ( ( -1 * Ky + Ly ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( \
    2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + ( \
    ( ( -1 * Iy + Ly ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + -2 * ( ( \
    -1 * Ix + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) * ( -1 * Ky + Ly \
    ) + ( ( -1 * Ix + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) * ( -1 * \
    Kz + Lz ) + ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) * \
    ( -1 * Kz + Lz ) ) ) ) ) ) ) ) ) )**( -1/2 ) ) )      
                                                                    
    ddLx = ( -1/2 * ( ( ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 \
    ) + ( ( ( -1 * Iz + Jz ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( \
    ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( ( ( -1 * \
    Iz + Jz ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( -2 * ( ( -1 * Ix \
    + Jx ) * ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( Jy + -1 * Ky ) + ( ( \
    -1 * Ix + Jx ) * ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( Jz + -1 * Kz \
    ) + ( -1 * Iy + Jy ) * ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( Jz + \
    -1 * Kz ) ) ) + ( ( ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jz + -1 * Kz ) \
    )**( 2 ) + ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) \
    ) ) ) ) ) ) )**( -1/2 ) * ( ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( \
    -1 * Kx + Lx ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Ix + Jx ) * ( Jy + \
    -1 * Ky ) * ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Iy + \
    Jy ) * ( Jx + -1 * Kx ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly ) + ( ( \
    -1 * Ix + Jx ) * ( Jy + -1 * Ky ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly \
    ) + ( ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( -1 * Kx + Lx ) * ( -1 * \
    Iz + Lz ) + ( -1 * ( -1 * Ix + Jx ) * ( Jz + -1 * Kz ) * ( -1 * Kx + \
    Lx ) * ( -1 * Iz + Lz ) + ( ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( \
    -1 * Ky + Ly ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iy + Jy ) * ( Jz + \
    -1 * Kz ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iz + \
    Jz ) * ( Jx + -1 * Kx ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz ) + ( ( \
    -1 * Ix + Jx ) * ( Jz + -1 * Kz ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz \
    ) + ( -1 * ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( -1 * Iy + Ly ) * ( \
    -1 * Kz + Lz ) + ( -1 * Iy + Jy ) * ( Jz + -1 * Kz ) * ( -1 * Iy + Ly \
    ) * ( -1 * Kz + Lz ) ) ) ) ) ) ) ) ) ) ) ) * ( 2 * ( -1 * Kx + Lx ) * \
    ( ( -1 * Iy + Ly ) )**( 2 ) + ( 2 * ( -1 * Ix + Lx ) * ( ( -1 * Ky + \
    Ly ) )**( 2 ) + ( 2 * ( -1 * Kx + Lx ) * ( ( -1 * Iz + Lz ) )**( 2 ) \
    + ( 2 * ( -1 * Ix + Lx ) * ( ( -1 * Kz + Lz ) )**( 2 ) + -2 * ( ( -1 \
    * Ix + Lx ) * ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) + ( ( -1 * Kx + Lx \
    ) * ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) + ( ( -1 * Ix + Lx ) * ( -1 * \
    Iz + Lz ) * ( -1 * Kz + Lz ) + ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) * \
    ( -1 * Kz + Lz ) ) ) ) ) ) ) ) * ( ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( \
    ( -1 * Iy + Ly ) )**( 2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * \
    Ky + Ly ) )**( 2 ) + ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iz + Lz \
    ) )**( 2 ) + ( ( ( -1 * Ky + Ly ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( \
    2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + ( \
    ( ( -1 * Iy + Ly ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + -2 * ( ( \
    -1 * Ix + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) * ( -1 * Ky + Ly \
    ) + ( ( -1 * Ix + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) * ( -1 * \
    Kz + Lz ) + ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) * \
    ( -1 * Kz + Lz ) ) ) ) ) ) ) ) ) )**( -3/2 ) + ( ( ( ( -1 * Iy + Jy ) \
    )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( -1 * Iz + Jz ) )**( 2 \
    ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( -1 * Ix + Jx ) )**( 2 ) * ( ( \
    Jy + -1 * Ky ) )**( 2 ) + ( ( ( -1 * Iz + Jz ) )**( 2 ) * ( ( Jy + -1 \
    * Ky ) )**( 2 ) + ( -2 * ( ( -1 * Ix + Jx ) * ( -1 * Iy + Jy ) * ( Jx \
    + -1 * Kx ) * ( Jy + -1 * Ky ) + ( ( -1 * Ix + Jx ) * ( -1 * Iz + Jz \
    ) * ( Jx + -1 * Kx ) * ( Jz + -1 * Kz ) + ( -1 * Iy + Jy ) * ( -1 * \
    Iz + Jz ) * ( Jy + -1 * Ky ) * ( Jz + -1 * Kz ) ) ) + ( ( ( -1 * Ix + \
    Jx ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) + ( ( -1 * Iy + Jy ) )**( \
    2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) ) ) ) ) ) ) )**( -1/2 ) * ( ( -1 * \
    Iy + Jy ) * ( Jx + -1 * Kx ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Ix + \
    Jx ) * ( Jy + -1 * Ky ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Iy + Jy ) \
    * ( Jx + -1 * Kx ) * ( -1 * Ky + Ly ) + ( ( -1 * Ix + Jx ) * ( Jy + \
    -1 * Ky ) * ( -1 * Ky + Ly ) + ( ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) \
    * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Ix + Jx ) * ( Jz + -1 * Kz ) * ( \
    -1 * Iz + Lz ) + ( -1 * ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( -1 * \
    Kz + Lz ) + ( -1 * Ix + Jx ) * ( Jz + -1 * Kz ) * ( -1 * Kz + Lz ) ) \
    ) ) ) ) ) ) * ( ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iy + Ly ) \
    )**( 2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * Ky + Ly ) )**( 2 \
    ) + ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( ( \
    ( -1 * Ky + Ly ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( ( ( -1 * \
    Ix + Lx ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + ( ( ( -1 * Iy + Ly \
    ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + -2 * ( ( -1 * Ix + Lx ) * \
    ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) + ( ( -1 * Ix \
    + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) * ( -1 * Kz + Lz ) + ( \
    -1 * Iy + Ly ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) * ( -1 * Kz + Lz \
    ) ) ) ) ) ) ) ) ) )**( -1/2 ) )      
                                                            
    ddLy = ( -1/2 * ( ( ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 \
    ) + ( ( ( -1 * Iz + Jz ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( \
    ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( ( ( -1 * \
    Iz + Jz ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( -2 * ( ( -1 * Ix \
    + Jx ) * ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( Jy + -1 * Ky ) + ( ( \
    -1 * Ix + Jx ) * ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( Jz + -1 * Kz \
    ) + ( -1 * Iy + Jy ) * ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( Jz + \
    -1 * Kz ) ) ) + ( ( ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jz + -1 * Kz ) \
    )**( 2 ) + ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) \
    ) ) ) ) ) ) )**( -1/2 ) * ( ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( \
    -1 * Kx + Lx ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Ix + Jx ) * ( Jy + \
    -1 * Ky ) * ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Iy + \
    Jy ) * ( Jx + -1 * Kx ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly ) + ( ( \
    -1 * Ix + Jx ) * ( Jy + -1 * Ky ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly \
    ) + ( ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( -1 * Kx + Lx ) * ( -1 * \
    Iz + Lz ) + ( -1 * ( -1 * Ix + Jx ) * ( Jz + -1 * Kz ) * ( -1 * Kx + \
    Lx ) * ( -1 * Iz + Lz ) + ( ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( \
    -1 * Ky + Ly ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iy + Jy ) * ( Jz + \
    -1 * Kz ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iz + \
    Jz ) * ( Jx + -1 * Kx ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz ) + ( ( \
    -1 * Ix + Jx ) * ( Jz + -1 * Kz ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz \
    ) + ( -1 * ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( -1 * Iy + Ly ) * ( \
    -1 * Kz + Lz ) + ( -1 * Iy + Jy ) * ( Jz + -1 * Kz ) * ( -1 * Iy + Ly \
    ) * ( -1 * Kz + Lz ) ) ) ) ) ) ) ) ) ) ) ) * ( 2 * ( ( -1 * Kx + Lx ) \
    )**( 2 ) * ( -1 * Iy + Ly ) + ( 2 * ( ( -1 * Ix + Lx ) )**( 2 ) * ( \
    -1 * Ky + Ly ) + ( 2 * ( -1 * Ky + Ly ) * ( ( -1 * Iz + Lz ) )**( 2 ) \
    + ( 2 * ( -1 * Iy + Ly ) * ( ( -1 * Kz + Lz ) )**( 2 ) + -2 * ( ( -1 \
    * Ix + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) + ( ( -1 * Ix + Lx \
    ) * ( -1 * Kx + Lx ) * ( -1 * Ky + Ly ) + ( ( -1 * Iy + Ly ) * ( -1 * \
    Iz + Lz ) * ( -1 * Kz + Lz ) + ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) * \
    ( -1 * Kz + Lz ) ) ) ) ) ) ) ) * ( ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( \
    ( -1 * Iy + Ly ) )**( 2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * \
    Ky + Ly ) )**( 2 ) + ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iz + Lz \
    ) )**( 2 ) + ( ( ( -1 * Ky + Ly ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( \
    2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + ( \
    ( ( -1 * Iy + Ly ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + -2 * ( ( \
    -1 * Ix + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) * ( -1 * Ky + Ly \
    ) + ( ( -1 * Ix + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) * ( -1 * \
    Kz + Lz ) + ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) * \
    ( -1 * Kz + Lz ) ) ) ) ) ) ) ) ) )**( -3/2 ) + ( ( ( ( -1 * Iy + Jy ) \
    )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( -1 * Iz + Jz ) )**( 2 \
    ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( -1 * Ix + Jx ) )**( 2 ) * ( ( \
    Jy + -1 * Ky ) )**( 2 ) + ( ( ( -1 * Iz + Jz ) )**( 2 ) * ( ( Jy + -1 \
    * Ky ) )**( 2 ) + ( -2 * ( ( -1 * Ix + Jx ) * ( -1 * Iy + Jy ) * ( Jx \
    + -1 * Kx ) * ( Jy + -1 * Ky ) + ( ( -1 * Ix + Jx ) * ( -1 * Iz + Jz \
    ) * ( Jx + -1 * Kx ) * ( Jz + -1 * Kz ) + ( -1 * Iy + Jy ) * ( -1 * \
    Iz + Jz ) * ( Jy + -1 * Ky ) * ( Jz + -1 * Kz ) ) ) + ( ( ( -1 * Ix + \
    Jx ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) + ( ( -1 * Iy + Jy ) )**( \
    2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) ) ) ) ) ) ) )**( -1/2 ) * ( -1 * ( \
    -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( -1 * Ix + Lx ) + ( ( -1 * Ix + \
    Jx ) * ( Jy + -1 * Ky ) * ( -1 * Ix + Lx ) + ( ( -1 * Iy + Jy ) * ( \
    Jx + -1 * Kx ) * ( -1 * Kx + Lx ) + ( -1 * ( -1 * Ix + Jx ) * ( Jy + \
    -1 * Ky ) * ( -1 * Kx + Lx ) + ( ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) \
    * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iy + Jy ) * ( Jz + -1 * Kz ) * ( \
    -1 * Iz + Lz ) + ( -1 * ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( -1 * \
    Kz + Lz ) + ( -1 * Iy + Jy ) * ( Jz + -1 * Kz ) * ( -1 * Kz + Lz ) ) \
    ) ) ) ) ) ) * ( ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iy + Ly ) \
    )**( 2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * Ky + Ly ) )**( 2 \
    ) + ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( ( \
    ( -1 * Ky + Ly ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( ( ( -1 * \
    Ix + Lx ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + ( ( ( -1 * Iy + Ly \
    ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + -2 * ( ( -1 * Ix + Lx ) * \
    ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) + ( ( -1 * Ix \
    + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) * ( -1 * Kz + Lz ) + ( \
    -1 * Iy + Ly ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) * ( -1 * Kz + Lz \
    ) ) ) ) ) ) ) ) ) )**( -1/2 ) )     
                                                            
    ddLz = ( -1/2 * ( ( ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 \
    ) + ( ( ( -1 * Iz + Jz ) )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( \
    ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( ( ( -1 * \
    Iz + Jz ) )**( 2 ) * ( ( Jy + -1 * Ky ) )**( 2 ) + ( -2 * ( ( -1 * Ix \
    + Jx ) * ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( Jy + -1 * Ky ) + ( ( \
    -1 * Ix + Jx ) * ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( Jz + -1 * Kz \
    ) + ( -1 * Iy + Jy ) * ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( Jz + \
    -1 * Kz ) ) ) + ( ( ( -1 * Ix + Jx ) )**( 2 ) * ( ( Jz + -1 * Kz ) \
    )**( 2 ) + ( ( -1 * Iy + Jy ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) \
    ) ) ) ) ) ) )**( -1/2 ) * ( ( -1 * Iy + Jy ) * ( Jx + -1 * Kx ) * ( \
    -1 * Kx + Lx ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Ix + Jx ) * ( Jy + \
    -1 * Ky ) * ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) + ( -1 * ( -1 * Iy + \
    Jy ) * ( Jx + -1 * Kx ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly ) + ( ( \
    -1 * Ix + Jx ) * ( Jy + -1 * Ky ) * ( -1 * Ix + Lx ) * ( -1 * Ky + Ly \
    ) + ( ( -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( -1 * Kx + Lx ) * ( -1 * \
    Iz + Lz ) + ( -1 * ( -1 * Ix + Jx ) * ( Jz + -1 * Kz ) * ( -1 * Kx + \
    Lx ) * ( -1 * Iz + Lz ) + ( ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( \
    -1 * Ky + Ly ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iy + Jy ) * ( Jz + \
    -1 * Kz ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) + ( -1 * ( -1 * Iz + \
    Jz ) * ( Jx + -1 * Kx ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz ) + ( ( \
    -1 * Ix + Jx ) * ( Jz + -1 * Kz ) * ( -1 * Ix + Lx ) * ( -1 * Kz + Lz \
    ) + ( -1 * ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( -1 * Iy + Ly ) * ( \
    -1 * Kz + Lz ) + ( -1 * Iy + Jy ) * ( Jz + -1 * Kz ) * ( -1 * Iy + Ly \
    ) * ( -1 * Kz + Lz ) ) ) ) ) ) ) ) ) ) ) ) * ( 2 * ( ( -1 * Kx + Lx ) \
    )**( 2 ) * ( -1 * Iz + Lz ) + ( 2 * ( ( -1 * Ky + Ly ) )**( 2 ) * ( \
    -1 * Iz + Lz ) + ( 2 * ( ( -1 * Ix + Lx ) )**( 2 ) * ( -1 * Kz + Lz ) \
    + ( 2 * ( ( -1 * Iy + Ly ) )**( 2 ) * ( -1 * Kz + Lz ) + -2 * ( ( -1 \
    * Ix + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) + ( ( -1 * Iy + Ly \
    ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) + ( ( -1 * Ix + Lx ) * ( -1 * \
    Kx + Lx ) * ( -1 * Kz + Lz ) + ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) * \
    ( -1 * Kz + Lz ) ) ) ) ) ) ) ) * ( ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( \
    ( -1 * Iy + Ly ) )**( 2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * \
    Ky + Ly ) )**( 2 ) + ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iz + Lz \
    ) )**( 2 ) + ( ( ( -1 * Ky + Ly ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( \
    2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + ( \
    ( ( -1 * Iy + Ly ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + -2 * ( ( \
    -1 * Ix + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) * ( -1 * Ky + Ly \
    ) + ( ( -1 * Ix + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) * ( -1 * \
    Kz + Lz ) + ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) * \
    ( -1 * Kz + Lz ) ) ) ) ) ) ) ) ) )**( -3/2 ) + ( ( ( ( -1 * Iy + Jy ) \
    )**( 2 ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( -1 * Iz + Jz ) )**( 2 \
    ) * ( ( Jx + -1 * Kx ) )**( 2 ) + ( ( ( -1 * Ix + Jx ) )**( 2 ) * ( ( \
    Jy + -1 * Ky ) )**( 2 ) + ( ( ( -1 * Iz + Jz ) )**( 2 ) * ( ( Jy + -1 \
    * Ky ) )**( 2 ) + ( -2 * ( ( -1 * Ix + Jx ) * ( -1 * Iy + Jy ) * ( Jx \
    + -1 * Kx ) * ( Jy + -1 * Ky ) + ( ( -1 * Ix + Jx ) * ( -1 * Iz + Jz \
    ) * ( Jx + -1 * Kx ) * ( Jz + -1 * Kz ) + ( -1 * Iy + Jy ) * ( -1 * \
    Iz + Jz ) * ( Jy + -1 * Ky ) * ( Jz + -1 * Kz ) ) ) + ( ( ( -1 * Ix + \
    Jx ) )**( 2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) + ( ( -1 * Iy + Jy ) )**( \
    2 ) * ( ( Jz + -1 * Kz ) )**( 2 ) ) ) ) ) ) ) )**( -1/2 ) * ( -1 * ( \
    -1 * Iz + Jz ) * ( Jx + -1 * Kx ) * ( -1 * Ix + Lx ) + ( ( -1 * Ix + \
    Jx ) * ( Jz + -1 * Kz ) * ( -1 * Ix + Lx ) + ( ( -1 * Iz + Jz ) * ( \
    Jx + -1 * Kx ) * ( -1 * Kx + Lx ) + ( -1 * ( -1 * Ix + Jx ) * ( Jz + \
    -1 * Kz ) * ( -1 * Kx + Lx ) + ( -1 * ( -1 * Iz + Jz ) * ( Jy + -1 * \
    Ky ) * ( -1 * Iy + Ly ) + ( ( -1 * Iy + Jy ) * ( Jz + -1 * Kz ) * ( \
    -1 * Iy + Ly ) + ( ( -1 * Iz + Jz ) * ( Jy + -1 * Ky ) * ( -1 * Ky + \
    Ly ) + -1 * ( -1 * Iy + Jy ) * ( Jz + -1 * Kz ) * ( -1 * Ky + Ly ) ) \
    ) ) ) ) ) ) * ( ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iy + Ly ) \
    )**( 2 ) + ( ( ( -1 * Ix + Lx ) )**( 2 ) * ( ( -1 * Ky + Ly ) )**( 2 \
    ) + ( ( ( -1 * Kx + Lx ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( ( \
    ( -1 * Ky + Ly ) )**( 2 ) * ( ( -1 * Iz + Lz ) )**( 2 ) + ( ( ( -1 * \
    Ix + Lx ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + ( ( ( -1 * Iy + Ly \
    ) )**( 2 ) * ( ( -1 * Kz + Lz ) )**( 2 ) + -2 * ( ( -1 * Ix + Lx ) * \
    ( -1 * Kx + Lx ) * ( -1 * Iy + Ly ) * ( -1 * Ky + Ly ) + ( ( -1 * Ix \
    + Lx ) * ( -1 * Kx + Lx ) * ( -1 * Iz + Lz ) * ( -1 * Kz + Lz ) + ( \
    -1 * Iy + Ly ) * ( -1 * Ky + Ly ) * ( -1 * Iz + Lz ) * ( -1 * Kz + Lz \
    ) ) ) ) ) ) ) ) ) )**( -1/2 ) )
    
    F_I = np.array([ddIx, ddIy, ddIz])
    F_J = np.array([ddJx, ddJy, ddJz])
    F_K = np.array([ddKx, ddKy, ddKz])
    F_L = np.array([ddLx, ddLy, ddLz])
    
    return np.array([F_I, F_J, F_K, F_L])

The above function calculates the bending forces that act upon the beads due to their positions in a unit cell.

In [47]:
A = np.array([0,0,0])
B = np.array([(np.sqrt(3)/2)*r0,r0/2,0])
C = np.array([0,r0,0])
D = np.array([-(np.sqrt(3)/2)*r0,r0/2,0])

flat_forces = unit_cell_bending_force(A,B,C,D)
print("In a flat configuration, the analytical forces returned are:\n {} \n on beads A,B,C,D respectively".format(flat_forces))

print("\n")
B += np.array([0,0,0.01])
disturbed_forces = unit_cell_bending_force(A,B,C,D)
print("After a disturbance to the position of B, the analytical forces returned are:\n {} \n on beads A,B,C,D respectively".format(disturbed_forces))



In a flat configuration, the analytical forces returned are:
 [[-2.22044605e-16  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-2.22044605e-16  0.00000000e+00  0.00000000e+00]
 [ 4.44089210e-16  0.00000000e+00  0.00000000e+00]] 
 on beads A,B,C,D respectively


After a disturbance to the position of B, the analytical forces returned are:
 [[ 6.15347943e-04  2.22044605e-16 -5.33049059e-02]
 [-1.23069589e-03  0.00000000e+00  5.32906951e-02]
 [ 6.15347943e-04 -2.22044605e-16 -5.33049059e-02]
 [ 0.00000000e+00  0.00000000e+00  5.33191168e-02]] 
 on beads A,B,C,D respectively


In [48]:
def bending_potential(A,B,C,D): 
    """
    For the unit cell ABCD where A and C are the central beads and B and D are
    the corner beads, this function returns the potential energy of the system
    given their positions.
    """
    n1 = np.cross((B-A),(B-C))
    n2 = np.cross((D-C),(D-A))
    
    n1mod = np.linalg.norm(n1)
    n2mod = np.linalg.norm(n2)
    
    n1hat = n1/n1mod
    n2hat = n2/n2mod
    
    V = 1-np.dot(n1hat,n2hat)
    
    return V

def finite_differencing_bending_force(displacement):
    
    # Initial positions (flat)
    A = np.array([0,0,0])
    B = np.array([(np.sqrt(3)/2)*r0,r0/2,0])
    C = np.array([0,r0,0])
    D = np.array([-(np.sqrt(3)/2)*r0,r0/2,0])
    
    # Apply displacement to B
    for i in range(3):
        B[i] = B[i] + displacement[i]
        
    V = bending_potential(A,B,C,D)
    
    epsilon = 1e-6
    
    # Forces on A
    A_prime = A + np.array([epsilon,0,0])
    V_prime = bending_potential(A_prime,B,C,D)
    F_x = (V_prime - V)/epsilon
    
    A_prime = A + np.array([0,epsilon,0])
    V_prime = bending_potential(A_prime,B,C,D)
    F_y = (V_prime - V)/epsilon

    A_prime = A + np.array([0,0,epsilon])
    V_prime = bending_potential(A_prime,B,C,D)
    F_z = (V_prime - V)/epsilon
    
    F_A = np.array([F_x, F_y, F_z])
    
    # Calculate forces on B
    B_prime = B + np.array([epsilon,0,0])
    V_prime = bending_potential(A,B_prime,C,D)
    F_x = (V_prime - V)/epsilon
    
    B_prime = B + np.array([0,epsilon,0])
    V_prime = bending_potential(A,B_prime,C,D)
    F_y = (V_prime - V)/epsilon

    B_prime = B + np.array([0,0,epsilon])
    V_prime = bending_potential(A,B_prime,C,D)
    F_z = (V_prime - V)/epsilon
    
    F_B = np.array([F_x, F_y, F_z])
    
    # Forces on C
    C_prime = C + np.array([epsilon,0,0])
    V_prime = bending_potential(A,B,C_prime,D)
    F_x = (V_prime - V)/epsilon
    
    C_prime = C + np.array([0,epsilon,0])
    V_prime = bending_potential(A,B,C_prime,D)
    F_y = (V_prime - V)/epsilon
    
    C_prime = C + np.array([0,0,epsilon])
    V_prime = bending_potential(A,B,C_prime,D)
    F_z = (V_prime - V)/epsilon
    
    F_C = np.array([F_x, F_y, F_z])
    
    # Forces on D
    D_prime = D + np.array([epsilon,0,0])
    V_prime = bending_potential(A,B,C,D_prime)
    F_x = (V_prime - V)/epsilon
    
    D_prime = D + np.array([0,epsilon,0])
    V_prime = bending_potential(A,B,C,D_prime)
    F_y = (V_prime - V)/epsilon

    D_prime = D + np.array([0,0,epsilon])
    V_prime = bending_potential(A,B,C,D_prime)
    F_z = (V_prime - V)/epsilon
    
    F_D = np.array([F_x, F_y, F_z])

    
    return np.array([F_A, F_B, F_C, F_D])

In [49]:
displacement = np.array([0,0,0.01])

finite_bending = finite_differencing_bending_force(displacement)
print("Via analytical differentiation:")
print(disturbed_forces)
print("\n")

print("Via finite differencing: ")
print(finite_bending)

Via analytical differentiation:
[[ 6.15347943e-04  2.22044605e-16 -5.33049059e-02]
 [-1.23069589e-03  0.00000000e+00  5.32906951e-02]
 [ 6.15347943e-04 -2.22044605e-16 -5.33049059e-02]
 [ 0.00000000e+00  0.00000000e+00  5.33191168e-02]]


Via finite differencing: 
[[ 6.15350104e-04  1.11022302e-10 -5.33022432e-02]
 [-1.23069166e-03  0.00000000e+00  5.32933555e-02]
 [ 6.15350104e-04  1.11022302e-10 -5.33022432e-02]
 [ 0.00000000e+00  0.00000000e+00  5.33217828e-02]]


As can be seen from the above results, all the non-zero forces are calculated correctly by the bending force analytical calculation. 

Important to note is that, clearly for the analytical solutions, there is computer error producing non-zero values for forces that should be zero. However, with the finite differencing method also displays a force for some values of the order of the epsilon used to perturb. As a result, we see here the 1e-10 result for F_C_y for example. This is actually a zero.

